In [7]:
import cv2
import pandas as pd
import numpy as np

In [3]:
vid = cv2.VideoCapture('./akn.036.020.left.segmented.avi')
ret, frame = vid.read()

In [27]:
cv2.imwrite('0.jpg', frame)

True

In [24]:
frame = cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0,255,0))

In [5]:
boxes = pd.read_csv('./akn.036.020.left.csv')

In [20]:
arr = np.round(np.asarray(boxes).astype(np.int))

In [21]:
rect = arr[0,1:5]

In [22]:
rect

array([779, 825, 339, 411])